### Recommendation graph model based on wrangled Santander bank customer & financial products data
Author: Mei Yong <br>
https://github.com/mei-yong/bank_product_recommendations

# Inital Setup

#### Install & import necessary Python packages

In [1]:
#!pip install py2neo
#!pip install python-igraph
#!pip install textblob

In [2]:
from py2neo import Graph #, Node, Relationship

#### Initialise graph db and test connection

In [3]:
# Initialise the graph db
uri = "bolt://localhost:7687"
user = "neo4j"
password = "password"
graph = Graph(uri=uri, user=user, password=password)

In [4]:
# Testing graph connection
query = "MATCH (n) RETURN COUNT(n) AS numberOfNodes"
graph.run(query).evaluate()

0

# Create the model in Neo4j
Note: CSV files need to be in the import folder if importing from a local location <br>
https://neo4j.com/developer/guide-import-csv/

#### Nodes

In [5]:
# Timing code execution
import time
import datetime
start_time = time.time()

In [6]:
# Create constraints on nodes
graph.run("CREATE CONSTRAINT ON (c:Customer) ASSERT c.id IS UNIQUE")
graph.run("CREATE CONSTRAINT ON (p:Product) ASSERT p.id IS UNIQUE")

In [7]:
# # Create new customer nodes taking into account constraints
# query = '''
# USING PERIODIC COMMIT 500
# LOAD CSV WITH HEADERS
# FROM "file:///santander_data_50k.csv" AS row
# MERGE (c:Customer {id: row.cust_id})
# ON CREATE SET c.first_contract_date = row.first_contract_date, c.household_gross_income = row.household_gross_income
# '''
# graph.run(query)

In [8]:
# Create new customer nodes taking into account constraints
query = '''
USING PERIODIC COMMIT 500
LOAD CSV WITH HEADERS
FROM "file:///santander_data_100k.csv" AS row
MERGE (c:Customer {id: row.cust_id})
'''
graph.run(query)

In [9]:
# Create new product nodes taking into account constraints
query = """
USING PERIODIC COMMIT 500
LOAD CSV WITH HEADERS
FROM "file:///santander_products.csv" AS row
MERGE (p:Product {id: row.prod_id})
ON CREATE SET p.product_type = row.product_type
"""
graph.run(query)

#### Relationships

In [10]:
# Create the edges between customers and the products they have
query = """
USING PERIODIC COMMIT 500
LOAD CSV WITH HEADERS FROM "file:///cust_prod_rels_100k.csv" AS row

WITH split(row.products, ';') AS products, row.cust_id AS cust_id
UNWIND products AS product_type

WITH DISTINCT cust_id, product_type
MATCH (c:Customer {id: cust_id})
MATCH (p:Product {product_type: product_type})
MERGE (c)-[:HAS_PRODUCT]->(p)
"""
graph.run(query)

In [11]:
# Timing code execution
execution_time = time.time() - start_time
print(f"Code execution time: {str(datetime.timedelta(seconds=execution_time))}")

Code execution time: 0:00:12.205829


# Query the db

In [20]:
# Find a few customer nodes
query = "MATCH(c:Customer) RETURN c LIMIT 5"
result = graph.run(query)

for row in result:
    print(row)

<Record c=(_0:Customer {id: '1380679'})>
<Record c=(_1:Customer {id: '914904'})>
<Record c=(_2:Customer {id: '890654'})>
<Record c=(_3:Customer {id: '262721'})>
<Record c=(_4:Customer {id: '1301032'})>


In [21]:
# Find a few product nodes
query = "MATCH(p:Product) RETURN p LIMIT 5"
result = graph.run(query)

for row in result:
    print(row)

<Record p=(_7689:Product {id: '1', product_type: 'savings_account'})>
<Record p=(_7690:Product {id: '2', product_type: 'guarantees'})>
<Record p=(_7691:Product {id: '3', product_type: 'current_account'})>
<Record p=(_7692:Product {id: '4', product_type: 'derivada_account'})>
<Record p=(_7693:Product {id: '5', product_type: 'payroll_account'})>


In [27]:
# Find a few relationships
query = """
MATCH (c:Customer)-->(p:Product) 
RETURN  c.id, p.product_type
LIMIT 5
"""

result = graph.run(query)

for row in result:
    print(row)

<Record c.id='72364' p.product_type='savings_account'>
<Record c.id='270730' p.product_type='savings_account'>
<Record c.id='27354' p.product_type='savings_account'>
<Record c.id='296457' p.product_type='savings_account'>
<Record c.id='473916' p.product_type='savings_account'>


In [ ]:
# # MY: don't run this, need to add properties to the customer nodes earlier on first

# # Find some customers who have products in common
# query = """
# MATCH (c:Customer)-->(p:Product)<--(otherCust)
# WHERE c.x CONTAINS "x"
# RETURN otherCust.id, COUNT(p) AS productsInCommon, COLLECT(p.product_type) AS productTypes
# ORDER BY productsInCommon DESC, otherCust.id
# LIMIT 10
# """
# result = graph.run(query)

# for row in result:
#     print(row)

In [ ]:
from igraph import Graph as IGraph